In [1]:
import threading
import subprocess
import asyncio
import time
import cv2
import nats
import numpy as np
from nats.aio.msg import Msg
import nest_asyncio
import random
from asyncio import CancelledError
from project.generated.project.common.proto.Inference_pb2 import InferenceList
from project.common import profiler
from project.common.config_class.profiler import ProfilerConfig
from project.generated.project.common.proto.Image_pb2 import ImageMessage
from project.generated.project.common.proto.Inference_pb2 import Inference
from project.common.camera.distance import StereoCameraConfig, calculate_distance_with_disparity, cameras_to_disparity_map
from project.common.config_class.camera_feed_cleaner import CameraConfig

nest_asyncio.apply()


# Function to draw bounding boxes
def render_detections(frame: np.ndarray, inference: Inference):
    for i in range(0, len(inference.bounding_box), 4):
        box = inference.bounding_box[i : i + 4]
        x1, y1, x2, y2 = map(int, box)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
        label = f"{inference.class_name}: {inference.confidence:.2f}"
        cv2.putText(
            frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
        )



def get_stereo_camera_config() -> StereoCameraConfig:
    K_left = [
        [1456.005797684768, 0.0, 922.3518652685229],
        [0.0, 1445.951965437783, 540.6621390641766],
        [0.0, 0.0, 1.0],
    ]
    D_left = [
        [
            -0.4503620891432703,
            0.2464259923048079,
            -0.0034755098126456395,
            0.0017071484321540516,
            -0.08767912738251105,
        ]
    ]
    K_right = [
        [1415.0937400593282, 0.0, 1000.8539866832261],
        [0.0, 1412.933279218525, 494.89189107717624],
        [0.0, 0.0, 1.0],
    ]
    D_right = [
        [
            -0.34241186296315873,
            0.13516356264093296,
            0.002027992770869345,
            0.002215580827335137,
            -0.035264585917374205,
        ]
    ]
    R = [
        [0.9586677652059092, -0.02374614118585852, 0.2835352477803932],
        [0.0019844362444381043, 0.9970450731076617, 0.07679312602394452],
        [-0.2845209622644691, -0.07305643688627948, 0.9558820947489192],
    ]
    T = [
        [-0.36803382859519196],
        [-0.005933201598225973],
        [0.051118031079258955],
    ]

    return StereoCameraConfig(K_left, D_left, K_right, D_right, R, T)


class ImageDetector:
    def __init__(self, nats_url="nats://localhost:4222"):
        self.nats_url = nats_url
        self.nats_client = None
        self.response_queue = asyncio.Queue()
        
    async def _setup(self):
        """Internal setup for NATS connection"""
        if not self.nats_client:
            self.nats_client = await nats.connect(self.nats_url)
            
            async def message_handler(msg):
                await self.response_queue.put(InferenceList.FromString(msg.data))
            
            await self.nats_client.subscribe(
                "recognition/image_output",
                cb=message_handler
            )

    async def detect(self, frame):
        """Process a single frame and return detections"""
        
        # Prepare and send the image
        _, compressed = cv2.imencode(".jpg", frame)
        image_id = random.randint(0, 1000000)
        
        msg = ImageMessage(
            image=compressed.tobytes(),
            camera_name="camera0",
            is_gray=False,
            id=image_id,
            height=frame.shape[0],
            width=frame.shape[1],
            timestamp=int(time.time() * 1000)
        )
        
        # Send image and wait for response
        await self.nats_client.publish("recognition/image_input", msg.SerializeToString())
        await self.nats_client.flush()
        inference_list = await self.response_queue.get()
        
        return inference_list.inferences

# Usage example:
async def main():
    # Initialize detector
    detector = ImageDetector()
    await detector._setup()
    stereo_camera_config = get_stereo_camera_config()
    
    # Initialize camera
    camera_left = cv2.VideoCapture(0)
    camera_right = cv2.VideoCapture(1)
    
    try:
        while True:
            ret_left, frame_left = camera_left.read()
            ret_right, frame_right = camera_right.read()
            if not ret_left or not ret_right:
                continue
                
            # Get detections
            detections = await detector.detect(frame_left)
            
            
            # Render detections (using your existing render_detections function)
            disparity, normalized_disparity, Q = cameras_to_disparity_map(
                left_img=frame_left,
                right_img=frame_right,
                stereo_camera_config=stereo_camera_config,
                do_rectify=True,
            )
            
            distance = 0
            for inference in detections:
                print(inference.bounding_box)
                distance = calculate_distance_with_disparity(
                    disparity=disparity,
                    Q=Q,
                    bbox=(int(inference.bounding_box[0]), int(inference.bounding_box[1]), int(inference.bounding_box[2]), int(inference.bounding_box[3])),
                )
                render_detections(frame_left, inference)
                
            # Display
            cv2.putText(frame_left, f"Distance: {distance:.2f} meters", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 255, 0), 2)
            cv2.imshow('frame', frame_left)
            cv2.imshow('disparity', disparity)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        camera_left.release()
        cv2.destroyAllWindows()

# Run it
nest_asyncio.apply()
await main()

2024-11-23 19:45:29.568 Python[87344:2131919] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-11-23 19:45:31.283 Python[87344:2131919] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


[1.44598388671875, 3.3376922607421875, 648.6046142578125, 725.7296752929688]
[337.0506591796875, 0.0, 900.1943359375, 381.6713562011719]
[343.5495300292969, 2.6475677490234375, 744.1912231445312, 466.32196044921875]
[1.9055328369140625, 2.9454345703125, 474.16717529296875, 1050.8768310546875]
[1.91015625, 2.945892333984375, 438.8612365722656, 1050.7005615234375]
[2.6560821533203125, 2.35894775390625, 519.4207763671875, 1053.843505859375]
[1.074920654296875, 1.76495361328125, 709.9175415039062, 1062.5789794921875]
[3.00860595703125, 2.519439697265625, 886.7864379882812, 1053.8848876953125]
[1.32421875, 2.2705078125, 913.496337890625, 1067.91064453125]
[36.854736328125, 2.80810546875, 905.870361328125, 1065.9898681640625]
[137.9088134765625, 1.3359375, 941.4188232421875, 1065.1055908203125]
[198.92410278320312, 1.127838134765625, 954.5260620117188, 1068.281005859375]
[253.02540588378906, 0.5511474609375, 948.5919799804688, 1063.9669189453125]
[292.40020751953125, 0.516357421875, 932.1112

TypeError: unsupported format string passed to NoneType.__format__